<br>

**This jupyter notebook touches on**
* **Set up GPU**
* **Push model and tensor to GPU for training**

<br>

## **Step 1: Import**

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

<br>

## **Step 2: Read**

In [2]:
dataset = pd.read_csv('step_2.csv')
dataset.head()

,no_pregnant,glucose,diastolic,skin_fold,insulin,bmi,pedigree,age,diabetic
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


<br>

## **Step 3: Assign features (X) and target (y)**

In [3]:
X = dataset.iloc[:, 0:8]
y = dataset.iloc[:, 8]

<br>

## **Step 4: Convert from Pandas dataframe to Numpy array**

In [4]:
X = X.to_numpy()
y = y.to_numpy()

In [5]:
X[:5]

array([[6.000e+00, 1.480e+02, 7.200e+01, 3.500e+01, 0.000e+00, 3.360e+01,
        6.270e-01, 5.000e+01],
       [1.000e+00, 8.500e+01, 6.600e+01, 2.900e+01, 0.000e+00, 2.660e+01,
        3.510e-01, 3.100e+01],
       [8.000e+00, 1.830e+02, 6.400e+01, 0.000e+00, 0.000e+00, 2.330e+01,
        6.720e-01, 3.200e+01],
       [1.000e+00, 8.900e+01, 6.600e+01, 2.300e+01, 9.400e+01, 2.810e+01,
        1.670e-01, 2.100e+01],
       [0.000e+00, 1.370e+02, 4.000e+01, 3.500e+01, 1.680e+02, 4.310e+01,
        2.288e+00, 3.300e+01]])

In [6]:
y[:5]

array([1, 0, 1, 0, 1], dtype=int64)

<br>

## **Step 5: Convert from Numpy array to PyTorch tensor**

In [7]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [8]:
X[:5]

tensor([[6.0000e+00, 1.4800e+02, 7.2000e+01, 3.5000e+01, 0.0000e+00, 3.3600e+01,
         6.2700e-01, 5.0000e+01],
        [1.0000e+00, 8.5000e+01, 6.6000e+01, 2.9000e+01, 0.0000e+00, 2.6600e+01,
         3.5100e-01, 3.1000e+01],
        [8.0000e+00, 1.8300e+02, 6.4000e+01, 0.0000e+00, 0.0000e+00, 2.3300e+01,
         6.7200e-01, 3.2000e+01],
        [1.0000e+00, 8.9000e+01, 6.6000e+01, 2.3000e+01, 9.4000e+01, 2.8100e+01,
         1.6700e-01, 2.1000e+01],
        [0.0000e+00, 1.3700e+02, 4.0000e+01, 3.5000e+01, 1.6800e+02, 4.3100e+01,
         2.2880e+00, 3.3000e+01]])

In [9]:
y[:5]

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.]])

<br>

## **Step 6: Set up GPU**

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


<br>

## **Step 7: Push tensor to GPU**

In [11]:
X = X.to(device)
y = y.to(device)

In [12]:
X[:5]

tensor([[6.0000e+00, 1.4800e+02, 7.2000e+01, 3.5000e+01, 0.0000e+00, 3.3600e+01,
         6.2700e-01, 5.0000e+01],
        [1.0000e+00, 8.5000e+01, 6.6000e+01, 2.9000e+01, 0.0000e+00, 2.6600e+01,
         3.5100e-01, 3.1000e+01],
        [8.0000e+00, 1.8300e+02, 6.4000e+01, 0.0000e+00, 0.0000e+00, 2.3300e+01,
         6.7200e-01, 3.2000e+01],
        [1.0000e+00, 8.9000e+01, 6.6000e+01, 2.3000e+01, 9.4000e+01, 2.8100e+01,
         1.6700e-01, 2.1000e+01],
        [0.0000e+00, 1.3700e+02, 4.0000e+01, 3.5000e+01, 1.6800e+02, 4.3100e+01,
         2.2880e+00, 3.3000e+01]], device='cuda:0')

In [13]:
y[:5]

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.]], device='cuda:0')

<br>

## **Step 8: Set up custom dataset**

In [14]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.y = y
        self.X = X
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        y = self.y[idx]
        X = self.X[idx]
        return X, y

In [15]:
df_dataset = CustomDataset(X, y)

In [16]:
k = 0
for i, j in df_dataset:
    print(i, '\t', j)
    k = k + 1
    if k == 5:
        break

tensor([  6.0000, 148.0000,  72.0000,  35.0000,   0.0000,  33.6000,   0.6270,
         50.0000], device='cuda:0') 	 tensor([1.], device='cuda:0')
tensor([ 1.0000, 85.0000, 66.0000, 29.0000,  0.0000, 26.6000,  0.3510, 31.0000],
       device='cuda:0') 	 tensor([0.], device='cuda:0')
tensor([  8.0000, 183.0000,  64.0000,   0.0000,   0.0000,  23.3000,   0.6720,
         32.0000], device='cuda:0') 	 tensor([1.], device='cuda:0')
tensor([ 1.0000, 89.0000, 66.0000, 23.0000, 94.0000, 28.1000,  0.1670, 21.0000],
       device='cuda:0') 	 tensor([0.], device='cuda:0')
tensor([  0.0000, 137.0000,  40.0000,  35.0000, 168.0000,  43.1000,   2.2880,
         33.0000], device='cuda:0') 	 tensor([1.], device='cuda:0')


<br>

## **Step 9: Set up custom dataloader with batch of 32**

In [17]:
df_dataloader = DataLoader(df_dataset, batch_size=32, shuffle=False)

In [18]:
k = 0
for i, j in df_dataloader:
    print(pd.DataFrame(zip(i, j), columns=['X','y']))
    k = k + 1
    if k == 2:   # print out 2 batches
        break

                                                    X  \
0   [tensor(6., device='cuda:0'), tensor(148., dev...   
1   [tensor(1., device='cuda:0'), tensor(85., devi...   
2   [tensor(8., device='cuda:0'), tensor(183., dev...   
3   [tensor(1., device='cuda:0'), tensor(89., devi...   
4   [tensor(0., device='cuda:0'), tensor(137., dev...   
5   [tensor(5., device='cuda:0'), tensor(116., dev...   
6   [tensor(3., device='cuda:0'), tensor(78., devi...   
7   [tensor(10., device='cuda:0'), tensor(115., de...   
8   [tensor(2., device='cuda:0'), tensor(197., dev...   
9   [tensor(8., device='cuda:0'), tensor(125., dev...   
10  [tensor(4., device='cuda:0'), tensor(110., dev...   
11  [tensor(10., device='cuda:0'), tensor(168., de...   
12  [tensor(10., device='cuda:0'), tensor(139., de...   
13  [tensor(1., device='cuda:0'), tensor(189., dev...   
14  [tensor(5., device='cuda:0'), tensor(166., dev...   
15  [tensor(7., device='cuda:0'), tensor(100., dev...   
16  [tensor(0., device='cuda:0'

<br>

## **Step 10: Set up model and push model to GPU**

In [19]:
# Hidden layer
#    Multi-Layer Perceptron = ReLU
#    Convolutional Neural Network = ReLU
#    Recurrent Neural Network = Sigmoid or Tanh

# Output layer
#    Regression = Linear
#    Binary = Sigmoid
#    Multi-class = Softmax
#    Multi-label = Sigmoid

model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid()
).to(device)
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


In [20]:
next(model.parameters()).is_cuda

True

<br>

## **Step 11: Train model**

In [21]:
loss_fn = nn.BCELoss()   # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)   # Learning rate = 0.001

n_epochs = 10

for epoch in range(n_epochs):
    for Xbatch, ybatch in df_dataloader:
        optimizer.zero_grad()
        y_pred = model(Xbatch)
        loss = loss_fn(y_pred, ybatch)
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')



Finished epoch 0, latest loss 0.9226192831993103
Finished epoch 1, latest loss 0.7865515351295471
Finished epoch 2, latest loss 0.7304971218109131
Finished epoch 3, latest loss 0.693583071231842
Finished epoch 4, latest loss 0.6659927368164062
Finished epoch 5, latest loss 0.6412714719772339
Finished epoch 6, latest loss 0.6319015026092529
Finished epoch 7, latest loss 0.6310555934906006
Finished epoch 8, latest loss 0.6312575340270996
Finished epoch 9, latest loss 0.6296596527099609


<br>

## **Step 12: Use trained model to compute accuracy on current data**
**Note that the actual data science workflow should split data into training and testing**

In [22]:
with torch.no_grad():
    y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.6822916865348816


In [23]:
print(y_pred[:5].round(), '\t', y[:5])

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.]], device='cuda:0') 	 tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.]], device='cuda:0')


In [24]:
print(y_pred[:5].round().cpu().detach().numpy(), '\t', y[:5].cpu().detach().numpy())

[[0.]
 [0.]
 [1.]
 [0.]
 [0.]] 	 [[1.]
 [0.]
 [1.]
 [0.]
 [1.]]
